In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"RAW/nhl_pbp_20212022.csv")


print(df.Event.unique())
#Filter events to only include SHOT, TAKE, BLOCK, MISS, GIVEE

['PSTR' 'FAC' 'HIT' 'STOP' 'SHOT' 'TAKE' 'BLOCK' 'MISS' 'GIVE' 'PEND'
 'GOAL' 'PENL' 'GEND' 'DELPEN' 'CHL' 'SOC' 'EISTR' 'EIEND' 'EGPID']
